In [ ]:
import sys, os
import pandas as pd
import netCDF4 as nc
import numpy as np
sys.path.append('../src/')
from Biologging_Toolkit.processing.Bioluminescence import Bioluminescence

#### Enter deployment ID and path

Finalized dataset will be saved or loaded from path

In [ ]:
depids = ['ml17_280a','ml18_296a','ml18_294b','ml19_292a','ml19_292b','ml19_293a','ml19_294a','ml20_293a','ml20_296b','ml20_313a','ml21_295a','ml21_305b']
depid = depids[9]
path = f'D:/individus_brut/individus/{depid}/'
sens_path = f'D:/individus_brut/CTD/{depid}/{depid}sens5.nc'
trk_path = f'D:/individus_brut/CTD/{depid}/{depid}trk.nc'

#### Run cells to get acoustic features

Time can be constructed from extisting time array or by entering the path to sens5 dataset.

In [ ]:
inst = Bioluminescence(depid,
            path = path,
            sens_path = sens_path,
            trk_path = trk_path
            )

In [ ]:
inst(overwrite = True)

In [ ]:
inst.get_flash()

In [ ]:
inst.ds.close()

### JOIN FLASH TO DIVE CSV (AMOUNT OF FLASH PER DIVE)

In [ ]:
for depid in depids[9:] :
    path = f'D:/individus_brut/individus/{depid}/'
    ds = nc.Dataset(os.path.join(path, f'{depid}_sens.nc'))
    df = pd.read_csv(os.path.join(path, f'{depid}_dive.csv'))
    sens_time = ds['time'][:].data
    try :
        LL = ds['flash'][:].data
    except IndexError :
        continue
    flash = []
    for i, row in df.iterrows():
        _LL = LL[(sens_time >= row.begin_time) & (sens_time <= row.end_time)]
        flash.append(np.sum(_LL[:,2]))
    df['flash'] = flash
    df.to_csv(os.path.join(path, f'{depid}_dive.csv'), index = None)

In [ ]:
ds.close()